Блокнот с решением должен иметь следующую структуру:

1. EDA (предварительный анализ данных): общее представление о данных, поиск и исправление ошибок, заполнение пропусков, анализ выбросов. Просьба не просто выводить цифры и графики, а писать понятные комментарии

2. Создание новых признаков и модификация существующих (если необходимо)

3. Применение модели

4. Анализ результатов: какие модели оказались лучше (если было больше 1-й), какие признаки оказали наибольшее влияние на результат

5. Улучшение качества (если есть необходимость): применение новых признаков, моделей, перебор гиперпараметров

Не забудьте сохранить файл sample_submission.csv, сохранить блокнот с решением (Save Version -> Save) и выполнить Submit решения (блокнот в режиме просмотра -> Output -> Submit)

После этого вы должны видеть свои результаты на LeaderBoard

Used https://www.kaggle.com/code/bertcarremans/data-preparation-exploration

# Считывание и подготовка данных для анализа

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/ubrr-ds/train.csv') # Train Data
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/ubrr-ds/test.csv') # Test Data
test.head()

In [ ]:
macro = pd.read_csv('/kaggle/input/ubrr-ds/macro.csv') # Additional data
macro.head()

In [ ]:
print("Train Shape: ",train.shape)
print("Test Shape: ",test.shape)
concat_df = pd.concat([train, test], axis=0)
print("Concat Shape: ",concat_df.shape)
concat_df.head()

In [ ]:
print("Concat keys: ", concat_df.columns.values)
print("Macro keys: ", macro.columns.values)
print("macro shape: ", macro.shape)
print("concat_ds shape: ", concat_df.shape)

In [ ]:
analyze_df = concat_df.merge(macro, on='timestamp', how='inner')
analyze_df.head()

# 1. EDA (предварительный анализ данных)

Подготовим данные для анализа

In [ ]:
analyze_df = concat_df.merge(macro, on='timestamp', how='inner')
analyze_df.head()

In [ ]:
analyze_df.info()

In [ ]:
analyze_df.describe()

In [ ]:
cols = analyze_df.columns[:30] # первые 30 колонок
# определяем цвета 
# желтый - пропущенные данные, синий - не пропущенные
colours = ['#000099', '#ffff00'] 
sns.heatmap(analyze_df[cols].isnull(), cmap=sns.color_palette(colours))

Рассмотрим NaN values для каждого столбца. Исправим значения, позже.

price_doc это наш target, поэтому можем не засчитывать его NaN как ошибку

In [ ]:
def analyze_nans():
    vars_with_missing = []

    for f in analyze_df.columns:
        missings = analyze_df[analyze_df[f].isna()].shape[0]
        if missings > 0:
            vars_with_missing.append(f)
            missings_perc = missings/analyze_df.shape[0]

            print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))

    print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))
analyze_nans()

* Рассмотрим столбец timestamp

In [ ]:
analyze_df.timestamp

Исправим тип столбца

In [ ]:
analyze_df.timestamp = pd.to_datetime(analyze_df.timestamp)
analyze_df.timestamp

* Рассмотрим столбцы full_sq, life_sq и kitch_sq отвечающие за размер квартиры

In [ ]:
show_columns = ['full_sq','life_sq','kitch_sq']
analyze_df[show_columns].describe()

Все 3 признака имеют выбросы, это можно заметить сравнив максимальные значения и 75% квартель. Также можно посмотреть на минимальное значение, размер не может равняться 0

In [ ]:
analyze_df[show_columns].info()

Колонки life_sq и kitch_sq содержат NaN

Заполним все NaN нулями

In [ ]:
analyze_df['life_sq'] = analyze_df['life_sq'].fillna(0.0)
analyze_df['kitch_sq'] = analyze_df['kitch_sq'].fillna(0.0)
analyze_df[show_columns].describe()

* Рассмотрим floor и max_floor

In [ ]:
analyze_df[['floor','max_floor']].dropna().sort_values(by='floor')

Имеются неверные данные. Этаж не может быть больше чем максимальный этаж.

In [ ]:
analyze_df[['floor','max_floor']].dropna().sort_values(by='floor').describe()

Видно выбросы. Разница между 75% и max слишком большая

Заполним na медианными значениями.

In [ ]:
analyze_df[['floor','max_floor']] = analyze_df[['floor','max_floor']].fillna(analyze_df[['floor','max_floor']].median())

* Рассмотрим state

Variable state has 3361 records (17.82%) with missing values

In [ ]:
analyze_df.state

In [ ]:
analyze_df.state.describe()

In [ ]:
analyze_df.state.value_counts()

In [ ]:
analyze_df.state.sort_values().dropna()

Ошибки в столбце: Большой процент NaN, Единичный выброс.

Заполним NaN и выброс v.

In [ ]:
analyze_df.state[analyze_df.state > 10] = analyze_df.state.quantile() 
analyze_df.state.fillna(analyze_df.state.mean())
analyze_df.state.describe()

In [ ]:
analyze_df.state.value_counts()

* Рассмотрим Material

In [ ]:
analyze_df.material

In [ ]:
analyze_df.material.value_counts()

In [ ]:
analyze_df.material.describe()

In [ ]:
analyze_df.material.info()

Ошибки: Есть NaN. заменим на 

2. Создание новых признаков и модификация существующих (если необходимо)

3. Применение модели

4. Анализ результатов: какие модели оказались лучше (если было больше 1-й), какие признаки оказали наибольшее влияние на результат


5. Улучшение качества (если есть необходимость): применение новых признаков, моделей, перебор гиперпараметров


Submission

In [ ]:
submission = pd.read_csv("/kaggle/input/ubrr-ds/submission.csv")
submission.head()

In [ ]:
# submission.to_csv("sample_submission.csv", index=False)